In [1]:
import sklearn
import pandas as pd
import json
import cufflinks as cf
import plotly.express as px
%matplotlib inline

In [2]:
rawResults = open('pg-analysis-results').read()

In [3]:
lines = rawResults.split('\n')

In [4]:
relevantLines = [line for line in lines if line.startswith('[')]

In [5]:
len(lines)

321

In [6]:
len(relevantLines)

159

In [7]:
cleaned = [line[:line.rfind(']')+1] for line in relevantLines]

In [8]:
parsed = [json.loads(line) for line in cleaned]

In [9]:
parsedCleaned = [item[0][0] for item in parsed]

In [10]:
df = pd.DataFrame(parsedCleaned)

In [11]:
df

,statsList,colorMapName,textName
0,"[{'parent': 'black', 'hex': '#1b2431', 'colorW...",XKCD,11
1,"[{'parent': 'black', 'hex': '#1b2431', 'colorW...",XKCD,11
2,"[{'parent': 'black', 'hex': '#000000', 'colorW...",XKCD,12
3,"[{'parent': 'black', 'hex': '#000000', 'colorW...",XKCD,13
4,"[{'parent': 'black', 'hex': '#000000', 'colorW...",XKCD,16
...,...,...,...
154,"[{'parent': 'black', 'hex': '#000000', 'colorW...",XKCD,572
155,"[{'parent': 'black', 'hex': '#000000', 'colorW...",XKCD,573
156,"[{'parent': 'black', 'hex': '#000000', 'colorW...",XKCD,574
157,"[{'parent': 'black', 'hex': '#000000', 'colorW...",XKCD,575


In [12]:
len(df.loc[0]['statsList'][0]['locations'])

3

In [13]:
stats = pd.DataFrame(df.loc[0]['statsList'])

In [14]:
df['stats'] = df['statsList'].apply(pd.DataFrame) 

In [15]:
def groupByParent(df): 
    try:
        return df.groupby('parent').sum()
    except:
        return

In [16]:
df['parentDist'] = df['stats'].apply(groupByParent)

In [17]:
df['parentDist'][0].to_dict()

{'nMatches': {'black': 3.461501183833405e-05,
  'blue': 1.384600473533362e-05,
  'green': 5.538401894133448e-05,
  'grey': 0.0004223031444276754,
  'orange': 1.384600473533362e-05,
  'purple': 2.769200947066724e-05,
  'red': 4.153801420600086e-05,
  'white': 0.00024922808523600515,
  'yellow': 0.000145383049721003}}

In [18]:
parentDists = {}
for i, row in df.iterrows():
    try:
        parentDists[row['textName']] = row['parentDist'].to_dict()['nMatches'] 
    except:
        continue

In [19]:
df2 = pd.DataFrame(parentDists)

In [20]:
df2 = df2.T

In [21]:
df2['name'] = df2.index

In [22]:
dfTidy = df2.melt(id_vars='name', var_name='color')

In [25]:
fig = px.bar(dfTidy, x='value', y='name', barmode='stack')

In [26]:
open('out.html', 'w').write(fig.to_html())

3371774

In [27]:
fig.show()